In [18]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import csv

# Open and read a CSV file
with open('xyz_ECOSTRESS.csv', mode='r') as file:
    reader = csv.reader(file)
    
    # Ignorer la première ligne (en-tête)
    next(reader)
    XYZ = np.array([[float(value) for value in row[1:]]  for row in reader])
    
#On récupère les colonnes X,Y,Z
X = XYZ[:, 0]
X_ = X.reshape(653, 1)
   
Y = XYZ[:, 1]
Y_ = Y.reshape(653, 1)
    
Z = XYZ[:, 2]
Z_ = Z.reshape(653, 1)

#On compile les 3 matrices
XYZ_ = np.vstack((X_, Y_, Z_))
    
    
with open('rho_ECOSTRESS_S2A.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    
    # Ignorer la première ligne (en-tête)
    next(csv_reader)
    
    # Ignorer la première colonne (en-tête)
    A_origin = np.array([[float(value) for value in row[1:]]  for row in csv_reader])
    
A_origin = A_origin[:,::-1]
A_x = A_origin.copy()
A_y = A_origin.copy()
A_z = A_origin.copy()

T = np.array([[0.371, 0.362,0.268 ],[0.174,0.587, 0.240],[-0.043,-0.427, 1.463]]).T



A_x[:, 0] *= T[0][0]  # Multiplier la colonne 1 par T[0][0] 
A_x[:, 1] *= T[1][0]   # Multiplier la colonne 2 par T[1][0] 
A_x[:, 2] *= T[2][0]   # Multiplier la colonne 3 par T[2][0] 

# print('T[:][0]',T[0][0],T[1][0],T[2][0])

A_y[:, 0] *= T[0][1]  
A_y[:, 1] *= T[1][1]  
A_y[:, 2] *= T[2][1] 

A_z[:, 0] *= T[0][2]  
A_z[:, 1] *= T[1][2]  
A_z[:, 2] *= T[2][2] 

#On compile les 3 matrices
A = np.vstack((A_x, A_y, A_z))

# print(A_x[0])
# print(A_origin[0]* T[0])

# print(A_origin[0:2,:])

P = np.eye(len(A))



def moindre_carre(B,A,P):
    N = (A.T)@P@A
    K = (A.T)@P@B
    X = np.linalg.inv(N)@K
    V = B - A @ X
    n = len(A)
    p = len(A[0])
    sigma_2 = V.T@P@V/(n-p)
    Q = (sigma_2)*(np.linalg.inv(A.T@P@A))
    return X, Q, sigma_2, V

def ecart_type(l):
    EcartT = 0
    n = len(l)
    moy = sum(l)/n
    for i in range(n):
         EcartT += (l[i] - moy)**2
    return sqrt(EcartT/(n-1))



X_sortie, Q, sigma_2, V = moindre_carre(XYZ_,A, P)
print('Constant :\n', X_sortie)
print('Résidu :\n', V)
print('sigma2 :\n', sigma_2)

#plt.scatter(A,Y, c = 'black', s = 5)
# y = X[0]*A_1 + X[1]
# plt.plot(A_1,y, c = 'red')
# plt.title("Régression linéaire DR1")
# plt.xlabel("Valeurs t")
# plt.ylabel("Valeurs de y")
# plt.text(-150,20, 'f"a = {X[0]}, b = {X[1]}')
# plt.show()

Constant :
 [[0.82709501]
 [1.01072311]
 [1.09444331]]
Résidu :
 [[-0.00082496]
 [ 0.00399062]
 [ 0.0096541 ]
 ...
 [-0.0483987 ]
 [-0.04642769]
 [-0.04749421]]
sigma2 :
 [[6.68803072e-05]]


In [19]:
# test
# D65 = np.array([[2.57511992],
#  [0.47153185],
#  [0.09773047]])
D65 = X_sortie.copy()
A_origin[:0] *=D65[0]
A_origin[:1] *=D65[1]
A_origin[:2] *=D65[2]
XYZ_cal = A_origin@T

In [20]:
#erreur MSE

# Calcul de l'erreur quadratique moyenne (MSE)
mse_X = np.mean((XYZ- XYZ_cal)**2)


# Racine carrée de l'erreur quadratique moyenne (RMSE)
rmse_X = np.sqrt(mse_X)


print(f"RMSE: {rmse_X}")

RMSE: 0.013985283788265489


In [21]:
#erreur R^2

from sklearn.metrics import r2_score
r_squared = r2_score(XYZ, XYZ_cal)



print(f"R^2: {r_squared}")

R^2: 0.979187779250371
